In [9]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LeakyReLU
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [10]:
# Generate synthetic regression data
X, y = make_regression(n_samples=1000, n_features=10, noise=0.1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
y_train[:10]

array([-259.74937779,  187.0425022 ,  -84.83987625, -137.84333754,
       -353.42655285,  255.59093858,  404.39309768, -443.00445418,
          9.45449075,  177.90173818])

In [12]:
# Standardize the data (important for neural networks)
scaler_X = StandardScaler()
X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)

In [13]:
y_train.shape

(800,)

In [14]:
scaler_y = StandardScaler()
y_train = scaler_y.fit_transform(y_train.reshape(-1, 1)).flatten() # (800, 1) instead of (800,)
y_test = scaler_y.transform(y_test.reshape(-1, 1)).flatten()  # (200, 1) instead of (200,)

In [15]:
y_train.shape

(800,)

In [16]:
X_train.shape[1]

10

In [17]:
# Build the ANN model
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))  # can be swish, tanh, sigmoid, linear, gelu, LeakyReLU(alpha=0.1)
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1))  # linear sum X*w Output layer for regression, no activation function

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')  # loss='mean_squared_error'  for regression

# Train the model
model.fit(X_train, y_train, epochs=300, batch_size=32, verbose=1, validation_split=0.2)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 0.9254 - val_loss: 0.8983
Epoch 2/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.8475 - val_loss: 0.6345
Epoch 3/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.4822 - val_loss: 0.2434
Epoch 4/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.2401 - val_loss: 0.0667
Epoch 5/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0478 - val_loss: 0.0334
Epoch 6/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0240 - val_loss: 0.0216
Epoch 7/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0142 - val_loss: 0.0153
Epoch 8/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0117 - val_loss: 0.0115
Epoch 9/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0078 - val_loss: 0.0106
Epoch 10/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0081 - val_loss: 0.0088
Epoch 11/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0065 - val_loss: 0.0092
Epoch 12/300
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.

In [18]:
# Evaluate the model on the test set
y_pred = model.predict(X_test)
print(y_pred.shape)
print(y_test.shape)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
(200, 1)
(200,)


In [19]:

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error on Test Set: {mse}")

# Inverse transform the predictions and actual values to their original scale
y_pred_original = scaler_y.inverse_transform(y_pred).flatten()
y_test_original = scaler_y.inverse_transform(y_test.reshape(-1, 1)).flatten()

# Print the first 5 predicted vs actual values
print("\nFirst 5 Predicted vs Actual values:")
for i in range(5):
    print(f"Predicted: {y_pred_original[i]:.2f}, Actual: {y_test_original[i]:.2f}")

Mean Squared Error on Test Set: 0.0021803429378929747

First 5 Predicted vs Actual values:
Predicted: -117.10, Actual: -111.78
Predicted: -52.40, Actual: -61.72
Predicted: -58.96, Actual: -64.83
Predicted: -144.99, Actual: -132.72
Predicted: -32.35, Actual: -49.57
